In [ ]:
import dask.dataframe as dd
import dask_geopandas as dg
import datetime as dt
import geopandas as gpd
import glob
import os
import pandas as pd
import sys

In [ ]:
DASK_GCS = "gs://calitp-analytics-data/data-analyses/dask_test/"
HQTA_GCS = "gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/"

PAIRWISE_FILE = f"{DASK_GCS}intermediate/pairwise.parquet"
SUBSET_CORRIDORS = f"{DASK_GCS}intermediate/subset_corridors.parquet"

In [ ]:
intersecting_pairs = pd.read_parquet(PAIRWISE_FILE)
corridors = gpd.read_parquet(SUBSET_CORRIDORS)

In [ ]:
metro = corridors[(corridors.calitp_itp_id == 182) & 
                  (corridors.route_id.str.contains("720"))]

In [ ]:
test_segment = 49694670 # a segment along the 720
this_segment = corridors[corridors.hqta_segment_id==test_segment]

In [ ]:
T = "Carto DB Positron"
this_segment.explore(tiles=T)

In [ ]:
these_are_intersecting_segments = intersecting_pairs[
    intersecting_pairs.hqta_segment_id==test_segment].intersect_hqta_segment_id.unique().tolist()

corridors[corridors.hqta_segment_id.isin(these_are_intersecting_segments)
         ].explore(tiles=T)


In [ ]:
test_intersect_segment = 1420142734
corresponding_pair = corridors[corridors.hqta_segment_id == test_intersect_segment][
    ["hqta_segment_id", "geometry"]]

corresponding_pair.explore(tiles=T)

In [ ]:
gpd.overlay(
    this_segment, 
    corresponding_pair, 
    how = "intersection", 
    keep_geom_type = False
).explore(tiles = T)

In [ ]:
gpd.clip(
    this_segment, 
    corresponding_pair, 
).explore(tiles=T)

In [ ]:
this_segment.geometry.iloc[0].intersection(corresponding_pair.geometry.iloc[0])

Now do an entire route.

In [ ]:
intersect_with_720 = intersecting_pairs[
    intersecting_pairs.hqta_segment_id.isin(metro.hqta_segment_id)].intersect_hqta_segment_id

In [ ]:
intersect_720_geom = corridors[corridors.hqta_segment_id.isin(intersect_with_720)][
    ["calitp_itp_id", "route_id", "hqta_segment_id", "geometry"]]

In [ ]:
metro.explore("hqta_segment_id", tiles=T)

In [ ]:
# Looping this still takes time...even with dask_geopandas.clip, because of the compute()
# Looking for a corresponding pair is ok..we know this already
# But can I try something else to get
gpd.clip(
    metro, 
    intersect_720_geom,
).explore(tiles=T)

In [ ]:
gpd.overlay(
    metro, 
    intersect_720_geom,
    how = "intersection"
).explore("hqta_segment_id_1", tiles=T)